In [8]:
from datetime import date, timedelta, datetime
import pandas as pd
from util import generate_quarters, saveDataToDatabase
import requests

In [9]:
api_key = "2b2bbacbc149bcba58903f591ae3d3c8"
market_url = "https://financialmodelingprep.com/api/v3/is-the-market-open"

In [10]:
params = {
    "apikey": api_key,
    }

In [11]:
response = requests.get(market_url, params=params)
data = response.json()

In [12]:
holidays=[]
for holiday_year in data['stockMarketHolidays']:
  for holiday_name, holiday_date in holiday_year.items():
    if holiday_name != 'year' and holiday_date is not None:
      holidays.append((datetime.strptime(str(holiday_date), "%Y-%m-%d"), 'close', holiday_name))

In [13]:
holidays_df = pd.DataFrame(holidays, columns=['dtDate', 'marketStatus', 'holiday'])

In [14]:
holidays_df.head()

,dtDate,marketStatus,holiday
0,2019-01-01,close,New Years Day
1,2019-01-21,close,"Martin Luther King, Jr. Day"
2,2019-02-18,close,Washington's Birthday
3,2019-04-19,close,Good Friday
4,2019-05-27,close,Memorial Day


In [15]:
date_range = pd.date_range(start='2019-01-01', end='2024-12-31', freq='D')

In [16]:
date_range_df = pd.DataFrame({'dtDate': date_range})
date_range_df.head()

,dtDate
0,2019-01-01
1,2019-01-02
2,2019-01-03
3,2019-01-04
4,2019-01-05


In [17]:
merge_df = pd.merge(date_range_df, holidays_df, how="left", on="dtDate")

In [18]:
merge_df['marketStatus'].fillna('open', inplace=True)

In [19]:
independence_day_mask = merge_df['holiday'] == 'Independence Day'
merge_df.loc[independence_day_mask.shift(-1).fillna(False), 'marketStatus'] = 'half'
merge_df.loc[independence_day_mask, 'marketStatus'] = 'close'

In [20]:
thanksgiving_day_mask = merge_df['holiday'] == 'Thanksgiving Day'
christmas_mask = merge_df['holiday'] == 'Christmas'
merge_df.loc[thanksgiving_day_mask.shift(1).fillna(False), 'marketStatus'] = 'half'
merge_df.loc[thanksgiving_day_mask, 'marketStatus'] = 'close'
merge_df.loc[christmas_mask.shift(1).fillna(False), 'marketStatus'] = 'half'
merge_df.loc[christmas_mask, 'marketStatus'] = 'close'

In [21]:
merge_df.loc[merge_df['dtDate'].dt.dayofweek.isin([5, 6]), 'marketStatus'] = 'close'

In [22]:
merge_df.drop('holiday', axis=1, inplace=True)

In [23]:
open_mask = merge_df['marketStatus'] == 'open'
merge_df.loc[open_mask, 'openingHour'] = data['stockMarketHours']['openingHour']
merge_df.loc[open_mask, 'closingHour'] = data['stockMarketHours']['closingHour']

In [24]:
half_mask = merge_df['marketStatus'] == 'half'
merge_df.loc[half_mask, 'openingHour'] = data['stockMarketHours']['openingHour']
merge_df.loc[half_mask, 'closingHour'] = "01:00 p.m. ET"

In [25]:
merge_df.tail(10)

,dtDate,marketStatus,openingHour,closingHour
2182,2024-12-22,close,NaN,NaN
2183,2024-12-23,open,09:30 a.m. ET,04:00 p.m. ET
2184,2024-12-24,open,09:30 a.m. ET,04:00 p.m. ET
2185,2024-12-25,close,NaN,NaN
2186,2024-12-26,half,09:30 a.m. ET,01:00 p.m. ET
2187,2024-12-27,open,09:30 a.m. ET,04:00 p.m. ET
2188,2024-12-28,close,NaN,NaN
2189,2024-12-29,close,NaN,NaN
2190,2024-12-30,open,09:30 a.m. ET,04:00 p.m. ET
2191,2024-12-31,open,09:30 a.m. ET,04:00 p.m. ET


In [26]:
merge_df.loc[open_mask, 'preMarket'] = '4:00am - 9:30am ET'
merge_df.loc[open_mask, 'postMarket'] = '4:00pm - 8:00pm ET'

In [27]:
merge_df["break"] = 0
merge_df["country"] = 'USA'

In [28]:
merge_df["stockExchange"] = data["stockExchangeName"]

In [29]:
merge_df

,dtDate,marketStatus,openingHour,closingHour,preMarket,postMarket,break,country,stockExchange
0,2019-01-01,close,NaN,NaN,NaN,NaN,0,USA,New York Stock Exchange
1,2019-01-02,open,09:30 a.m. ET,04:00 p.m. ET,4:00am - 9:30am ET,4:00pm - 8:00pm ET,0,USA,New York Stock Exchange
2,2019-01-03,open,09:30 a.m. ET,04:00 p.m. ET,4:00am - 9:30am ET,4:00pm - 8:00pm ET,0,USA,New York Stock Exchange
3,2019-01-04,open,09:30 a.m. ET,04:00 p.m. ET,4:00am - 9:30am ET,4:00pm - 8:00pm ET,0,USA,New York Stock Exchange
4,2019-01-05,close,NaN,NaN,NaN,NaN,0,USA,New York Stock Exchange
...,...,...,...,...,...,...,...,...,...
2187,2024-12-27,open,09:30 a.m. ET,04:00 p.m. ET,4:00am - 9:30am ET,4:00pm - 8:00pm ET,0,USA,New York Stock Exchange
2188,2024-12-28,close,NaN,NaN,NaN,NaN,0,USA,New York Stock Exchange
2189,2024-12-29,close,NaN,NaN,NaN,NaN,0,USA,New York Stock Exchange
2190,2024-12-30,open,09:30 a.m. ET,04:00 p.m. ET,4:00am - 9:30am ET,4:00pm - 8:00pm ET,0,USA,New York Stock Exchange


In [30]:
merge_df["url"] = "https://site.financialmodelingprep.com/developer/docs/"

In [31]:
merge_df.to_csv("marketCalender.csv", index=False)

In [32]:
from datetime import datetime

In [33]:
start_date = datetime.strptime("2022-2-2", "%Y-%m-%d")
start_date

datetime.datetime(2022, 2, 2, 0, 0)

In [34]:
start_date = start_date.replace(month=1, day=1)
start_date

datetime.datetime(2022, 1, 1, 0, 0)